In [1]:
import psycopg2
#Psycopg is the most popular PostgreSQL database adapter for the Python programming language. 

from config import USER, PASSWORD
#PostgresQL credentials

# First let's create a connection to the database

This connects to our local instance of PostgreSQL. From there we will use the database/schema. This connection will reach out to the database and insure we hvae the correct privilages to connect to this database.

First let's create a connection to the database
This connects to our local instance of PostgreSQL. From there we will use the database/schema. This connection will reach out to the database and insure we hvae the correct privilages to connect to this database.

In [2]:
try:
    conn = psycopg2.connect(f"host=127.0.0.1 dbname=udacity user={USER} password={PASSWORD}")
except psycopg2.Error as e:
    print("Error:Could not make connection to the Postgres Database")
    print(e)

In [3]:
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error:Could not get cursor to the Database")
    print(e)

In [4]:
conn.set_session(autocommit=True)

## Let's Test our Connection and our Error Handling

We are trying to do a select * on a table we have not created yet. We should expect to see a nicely handled error.

In [5]:
try:
    cur.execute("select * from music_library;")
except psycopg2.Error as e:
    print(e)

relation "music_library" does not exist
LINE 1: select * from music_library;
                      ^



### Let's imagene we would like to start creating a Music Library of albums. Each album has a lot of information we could add to them music library table, but we will just start with album name, artist name, year. The type of table we want to create and the columns we need seems to fall out very naturally.

> Table name: music_library <br>
> column 1: Album Name <br>
> column 2: Artist Name <br>
> column 3: Year

### Now to translate this information into a Create Table Statement.


In [6]:
try: cur.execute("CREATE TABLE IF NOT EXISTS music_library (album_name varchar, artist_name varchar, year int);")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

### No error was found, but lets check to ensure our table was created. select count(*) which should return 0 as we have not inserted any rows.

In [7]:
try: cur.execute("SELECT count(*) from music_library;")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

print(cur.fetchall())

[(0,)]


### Let's insert two rows
Note the syntax here

In [8]:
try:
    cur.execute("INSERT INTO music_library (album_name, artist_name, year) \
                VALUES (%s, %s, %s)",\
               ("Let it Be", "The beatles", 1970))
    
except psycopg2.Error as e:
    print("Error: inserting Rows")
    print(e)
    
try:
    cur.execute("INSERT INTO music_library (album_name, artist_name, year) \
                VALUES (%s, %s, %s)",\
               ("Robber Soul", "The beatles", 1965))
    
except psycopg2.Error as e:
    print("Error: inserting Rows")
    print(e)

### Validate your data was inserted into the table.
Note: The while loop is used for printing the results. If execution queries in the Postgres shell, this would not be requiered.

In [9]:
try:
    cur.execute("SELECT * FROM music_library;")
except psycopg2.Error as e:
    print("Error: select *")
    print(e)
    
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

('Let it Be', 'The beatles', 1970)
('Robber Soul', 'The beatles', 1965)


###  Create a Song Library that contains a list of songs, including the song name, artist name, year, album it was from, and if it was a single.

> column 1: song_title <br>
> column 2: artist_name <br>
> column 3: year <br>
> column 4: album_name <br>
> column 5: single

### Now to translate this information into a Create Table Statement.

In [11]:
try:
    cur.execute("CREATE TABLE songs (song_title varchar, artist_name varchar, year int, album_name varchar, single Boolean);")
except psycopg2.Error as e:
    print("Error: issue creating table")
    print(e)

### Let's insert two rows

> First row: "Across The universe", "The Beatles", "1970", "False", "Let it Be" <br>
> Second row: "The Beatles", "Think For Yourself", "False", "1965", "Rubber Soul"

In [12]:
try:
    cur.execute("INSERT INTO songs (song_title, artist_name, year, album_name, single) \
                VALUES (%s, %s, %s, %s, %s)", \
                ("Across The universe", "The Beatles", 1970, "Let it Be", False))
except psycopg2.Error as e:
    print("Error: Inserting Rows")
    print(e)
                 
try:
    cur.execute("INSERT INTO songs (song_title, artist_name, year, album_name, single) \
                VALUES (%s, %s, %s, %s, %s)", \
                ("Think For Yourself", "The Beatles", 1965, "Rubber Soul", False))
except psycopg2.Error as e:
    print("Error: Inserting Rows")
    print(e)               

### Check if the rows was created

In [13]:
try:
    cur.execute("SELECT * FROM songs;")
except psycopg2.Error as e:
    print("Error: select *")
    print(e)
    
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

('Across The universe', 'The Beatles', 1970, 'Let it Be', False)
('Think For Yourself', 'The Beatles', 1965, 'Rubber Soul', False)


### For the sake of the exercise, I will drop the tables.

In [14]:
try:
    cur.execute("DROP table music_library; DROP table songs;")
except psycopg2.Error as e:
    print("Error: Dropping table")
    print(e)

### And finally close your cursor and connection.

In [15]:
cur.close()
conn.close()